In [97]:
import pandas as pd
import numpy as np

In [98]:
products = pd.read_csv(r"C:\Users\Admin\Documents\Coursera Courses\Machine Learning - Classification\Week 2\amazon_baby_subset.csv")

In [87]:
products.head()[:2]

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1


In [88]:
products.head(10)

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1
5,Our Baby Girl Memory Book,"Beautiful book, I love it to record cherished ...",5,1
6,Hunnt&reg; Falling Flowers and Birds Kids Nurs...,"Try this out for a spring project !Easy ,fun a...",5,1
7,Blessed By Pope Benedict XVI Divine Mercy Full...,very nice Divine Mercy Pendant of Jesus now on...,5,1
8,Cloth Diaper Pins Stainless Steel Traditional ...,We bought the pins as my 6 year old Autistic s...,4,1
9,Cloth Diaper Pins Stainless Steel Traditional ...,It has been many years since we needed diaper ...,5,1


In [89]:
important_words = pd.read_json(r"C:\Users\Admin\Documents\Coursera Courses\Machine Learning - Classification\Week 2\important_words.json")

In [99]:
products = products.fillna("")

In [100]:
def remove_punctuation(line):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return line.translate(translator)

In [101]:
products["review_clean"] = products["review"].apply(remove_punctuation)

In [102]:
for word in important_words[0]:
    products[word] = products["review_clean"].apply(lambda x: x.split().count(word))

In [115]:
np.sum(products["perfect"] >= 1)

2955

In [118]:
features = ["constant"] + list(important_words[0])
# products.insert(0,"constant",1)
feature_matrix = products.as_matrix([features])

In [113]:
sentiment = products["sentiment"].values

In [119]:
feature_matrix.shape

(53072, 194)

In [186]:
####produces probablistic estimate for P(y_i = +1 | x_i, w).estimate ranges between 0 and 1.
def predict_probability(feature_matrix, coefficients):
    scores = np.dot(feature_matrix,coefficients)
    P = 1.0/(1.0+np.exp(-scores))
    return P

In [187]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors,feature)
    # Return the derivative
    return derivative

In [188]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

In [211]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    #Initialize coefficients
    coefficients = np.array(initial_coefficients)
    #Predict the class probability P(yi=+1|xi,w) using your predict_probability function and save it to variable predictions.
    for itr in list(range(max_iter)):
        predictions = predict_probability(feature_matrix, coefficients)
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        #error
        errors = indicator - predictions
        for j in list(range(len(coefficients))):
            derivative = feature_derivative(errors, feature_matrix[:,j])
            coefficients[j] = coefficients[j] + (step_size*derivative)
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:    
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [212]:
# initial_coefficients = [0 for i in list(range(0,194))]
initial_coefficients = np.zeros(194)

step_size = 1e-7
max_iter = 301
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [214]:
# coefficients
scores = np.dot(feature_matrix,coefficients)

In [226]:
predicted_sentiments = np.array([1 if s>0 else -1 for s in scores])

In [228]:
print(sum(predicted_sentiments==1))

25126


In [230]:
sum(predicted_sentiments==sentiment)

39903

In [233]:
num_mistakes = sum(predicted_sentiments!=sentiment)
accuracy = 1-float(num_mistakes)/len(sentiment)
print ("-----------------------------------------------------")
print ('# Reviews   correctly classified =', len(products) - num_mistakes)
print ('# Reviews incorrectly classified =', num_mistakes)
print ('# Reviews total                  =', len(products))
print ("-----------------------------------------------------")
print ('Accuracy = %.2f' % accuracy)


-----------------------------------------------------
# Reviews   correctly classified = 39903
# Reviews incorrectly classified = 13169
# Reviews total                  = 53072
-----------------------------------------------------
Accuracy = 0.75


In [255]:
print([features[s] for s in np.argsort(coefficients)])


['would', 'product', 'money', 'work', 'even', 'disappointed', 'get', 'back', 'return', 'monitor', 'waste', 'thought', 'could', 'tried', 'way', 'time', 'buy', 'item', 'returned', 'broke', 'better', 'plastic', 'reviews', 'cheap', 'thing', 'hard', 'first', 'never', 'something', 'bad', 'received', 'company', 'bought', 'idea', 'working', 'unit', 'another', 'try', 'going', 'box', 'two', 'piece', 'difficult', 'got', 'completely', 'trying', 'worked', 'cup', 'open', 'stay', 'milk', 'one', 'away', 'picture', 'times', 'design', 'weeks', 'made', 'anything', 'bottom', 'either', 'ordered', 'part', 'top', 'pump', 'quality', 'went', 'said', 'tub', 'right', 'problem', 'maybe', 'water', 'instead', 'started', 'child', 'less', 'actually', 'last', 'buying', 'new', 'make', 'however', 'came', 'different', 'bottle', 'gate', 'fit', 'purchased', 'see', 'much', 'took', 'know', 'wanted', 'say', 'give', 'like', 'want', 'together', 'place', 'small', 'head', 'second', 'come', 'getting', 'month', 'amazon', 'find', 's

In [250]:
np.argsort(coefficients)[:]

array([  6,  19,  97,  78,  33, 106,  13,  29, 114,  98, 113, 100,  47,
        99,  67,  17,  50, 102, 169, 172,  70, 104, 107, 134,  69, 103,
        32, 101, 108, 140, 123, 167,  18, 176, 181, 183,  94, 122, 109,
       157,  37, 156, 160,  39, 186, 159, 135, 112, 130, 174, 139,   2,
       132, 185, 125, 146, 148,  53, 165, 131, 193, 137, 141, 117, 110,
        68, 180, 178, 191,  80, 119, 170, 121, 175, 164,  54, 179, 133,
       166, 188,  73,  61, 105, 138, 116, 111, 129,  40,  79,  92,  22,
       171, 120, 150, 149, 163,   7, 118, 161, 152,  57, 177, 155, 168,
       136,  44, 115, 127, 124, 151,  26, 128, 192, 142,  82,  38, 126,
        77,  96, 182, 154,  46, 190,  15, 143,  30, 144,  25, 153,  93,
       147,  49,  85,  64,  90, 158, 173, 145,  20,  89, 184, 189,  36,
        86,  43,  74, 162,  10, 187,  42,  60,   5,   0,  75,  72,  59,
        71,  65,  55,  62,  58,  41,  81,  28,  87,  63,  95,  45,  51,
        24,  56,  52,  88,  16,  66,  27,  84,  76,  14,  31,   

In [240]:
min(coefficients)

-0.053860148445203135